In [21]:
import numpy as np 
import pandas as pd
from tqdm import tqdm

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
from retinanet import paps_eval
from retinanet import paps_train

# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.anchors import *
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.hourglass import hg1, hg2, hg8
from retinanet.parallel import DataParallelModel, DataParallelCriterion

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim

# python train_paps.py --start_epoch 0 --end_epoch 120 --batch_size 24 \
#                     --saved_dir $OUT_MODEL_DIR --gpu_num 0 --num_workers 12 \
#                     --target_threshold 7 --topk 20 --filter_option 1  | 2>&1 | tee $log


In [2]:
# class Anchors(nn.Module):
#     def __init__(self, pyramid_levels=None, strides=None, sizes=None, ratios=None, scales=None):
#         super(Anchors, self).__init__()

#         if pyramid_levels is None:
#             self.pyramid_levels = [3]
#         if strides is None:
#             self.strides = [2 ** x for x in self.pyramid_levels]
#         if sizes is None:
#             self.sizes = [2 ** (x + 2) for x in self.pyramid_levels]
#         if ratios is None:
#             self.ratios = np.array([0.5, 0.75, 1, 1.25, 1.5])
# #             self.ratios = np.array([1])
#         if scales is None:
# #             self.scales = np.array([2 ** 0])
#             self.scales = np.array([2 ** (-1.0 / 3.0), 2 ** (-3.0 / 3.0), 2 ** 0, 2 ** (1.0 / 3.0) ])

#     def forward(self, image):
        
#         image_shape = image.shape[2:]
#         image_shape = np.array(image_shape)
#         image_shapes = [(image_shape + 2 ** x - 1) // (2 ** x) for x in self.pyramid_levels]

#         # compute anchors over all pyramid levels
#         all_anchors = np.zeros((0, 4)).astype(np.float32)

#         for idx, p in enumerate(self.pyramid_levels):
#             anchors         = generate_anchors(base_size=self.sizes[idx], ratios=self.ratios, scales=self.scales)
#             shifted_anchors = shift(image_shapes[idx], self.strides[idx], anchors)
#             all_anchors     = np.append(all_anchors, shifted_anchors, axis=0)

#         all_anchors = np.expand_dims(all_anchors, axis=0)

#         if torch.cuda.is_available():
#             return torch.from_numpy(all_anchors.astype(np.float32)).cuda()
#         else:
#             return torch.from_numpy(all_anchors.astype(np.float32))

In [3]:
# anchors = Anchors()(torch.randn([1,3,320,320]))
# anchors.shape

In [22]:
# device = torch.device('cpu')
# device = torch.device('cuda')
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
model = hg2(device, pretrained=True, progress=False, num_classes=2)
model.to(device)

num_classes 2
num_anchors per feature map 20


HourglassNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
  )
  (layer2): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=Tr

In [5]:
# model

In [23]:

# train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

batch_size = 32
dataset_train = PapsDataset('../data/', set_name='train_2class',
                            transform=train_transforms)

train_data_loader = DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=12,
#     pin_memory=True,
#     prefetch_factor=1,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


In [24]:
criterion = PapsLoss(device, target_threshold=8, topk=10, filter_option=4)
criterion = criterion.to(device)
model.training = True

# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(model.parameters(), lr = 1e-8)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, T_mult=2, eta_max=0.0008,  T_up=5, gamma=0.5)
# CosineAnnealingWarmRestarts

In [8]:
saved_dir = '../trained_models/HourGlass/anchor' + str(GPU_NUM) + '/'
s_epoch= 0
e_epoch= 60

In [9]:
if os.path.isdir(saved_dir) == False :
    print('folder is made')
    os.makedirs(saved_dir)

In [25]:
if os.path.isfile(saved_dir + 'epoch_' + str(e_epoch) +'_model.pt') :
    print('pretrainind file loading')
    state = torch.load(saved_dir + 'epoch_' + str(e_epoch) +'_model.pt')
    epoch = state['epoch']
    model.load_state_dict(state['state_dict'], strict=False)
    optimizer.load_state_dict(state['optimizer'])
    last_loss = state['loss']
else :
    last_loss = 0.6

pretrainind file loading


In [11]:
paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  

  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                           

0th epochs loss 1.1313291513791648 lr 1e-08 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                

1th epochs loss 9.509377655055788 lr 0.000160008 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                 

...

  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                              

57th epochs loss 0.19929809314318192 lr 8.0006e-05 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                 

58th epochs loss 0.1969459447532128 lr 0.000120004 


59th epochs loss 0.19658905299555543 lr 0.00016000200000000002 


In [12]:
# state = {
#     'epoch': 0,
#     'state_dict': model.state_dict(),
#     'optimizer': optimizer.state_dict(),
#     'loss' : 0.6
# }
# torch.save(state, saved_dir + 'model.pt')

In [13]:
dataset_val = PapsDataset('../data/', set_name='val_2class',
                            transform=val_transforms)

val_data_loader = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


In [14]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=model, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.1) 


100%|██████████| 3739/3739 [02:22<00:00, 26.27it/s]


GT_results 17716
pred_results 158733
validation time : 144.18783354759216
Loading and preparing results...
DONE (t=0.73s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!
******************total*********************
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=18.28s).
Accumulating evaluation results...
DONE (t=0.82s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.324
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.110
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.166
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.060
 Average Recall     (AR) @

In [15]:
criterion = PapsLoss(device, target_threshold=9, topk=10, filter_option=4)
criterion = criterion.to(device)
s_epoch= 60
e_epoch= 120

paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  

  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                

60th epochs loss 0.18369287454312846 lr 0.0002 
best model is saved


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                

61th epochs loss 0.17658002477171075 lr 0.0001998629603277836 
best model is saved


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                  

...

  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                    

117th epochs loss 0.13466660133921182 lr 1.2411043821892097e-06 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                  

118th epochs loss 0.13306032735695186 lr 5.577830726495014e-07 


119th epochs loss 0.1302350768739851 lr 1.4703967221638957e-07 
best model is saved


In [16]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=model, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.1) 

100%|██████████| 3739/3739 [02:23<00:00, 26.13it/s]


GT_results 17716
pred_results 92560
validation time : 144.30821776390076
Loading and preparing results...
DONE (t=0.48s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
******************total*********************
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=14.49s).
Accumulating evaluation results...
DONE (t=0.58s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.179
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.481
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.168
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.213
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.069
 Average Recall     (AR) @[

In [27]:
criterion = PapsLoss(device, target_threshold=9, topk=10, filter_option=4)
criterion = criterion.to(device)
s_epoch= 120
e_epoch= 180

paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  

  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                   

120th epochs loss 0.1106516803785037 lr 4.4834866740579426e-05 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                  

121th epochs loss 0.11537118026843438 lr 0.000160008 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                  

...

134th epochs loss 0.13742006063843384 lr 0.0002763997473349929 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                 

135th epochs loss 0.1360543807092895 lr 0.00020000750000000007 


 32%|███▏      | 151/465 [02:20<04:15,  1.23it/s, avg_loss=0.13, batch_loss=0.0947, cls_loss=0.0911, lr=0.000132, num_det=tensor(592, device='cuda:0'), reg_loss=0.00361]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa4bed8c3b0>
Traceback (most recent call last):
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1297, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/multiprocessing/popen_fork.py", line 45, in wait
    if not wait([self.sentinel], timeout):
  File "/home/beomgon/anaconda3/envs/p

KeyboardInterrupt: 

In [ ]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=model, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.1) 

In [ ]:
criterion = PapsLoss(device, target_threshold=9, topk=10, filter_option=4)
criterion = criterion.to(device)
s_epoch= 180
e_epoch= 300

paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  

In [ ]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=model, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.1) 